In [2]:
### 로컬 데이터 불러오기 ###
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'. format(name = fn, length = len(uploaded[fn])))

Saving CreditCard.csv to CreditCard (1).csv
User uploaded file "CreditCard.csv" with length 77138 bytes


In [0]:
### 모듈 및 함수 로딩 ###
import io
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import random
from sklearn.preprocessing import LabelEncoder, normalize

In [0]:
### 데이터 로딩 ###
credit_card = pd.read_csv(io.StringIO(uploaded['CreditCard.csv'].decode('utf-8')))

In [0]:
### 범주형 변수 더미화 ###
credit_cat = credit_card[["card", "owner", "selfemp"]]

label_enc = LabelEncoder()

card = label_enc.fit_transform(credit_cat["card"])
card.shape = (len(card), 1)

owner = label_enc.fit_transform(credit_cat["owner"])
owner.shape = (len(owner), 1)

selfemp = label_enc.fit_transform(credit_cat["selfemp"])
selfemp.shape = (len(selfemp), 1)

In [0]:
### 수치형 변수 정규화 ###
credit_num  = credit_card.drop(["card", "owner", "selfemp", "share"], axis = 1)
credit_num_norm = normalize(credit_num)

In [0]:
### X, y 데이터 설정 ###
credit_X = np.concatenate([card, owner, selfemp, credit_num_norm], axis = 1)
credit_y = np.array(credit_card['share'])
credit_y.shape = (len(credit_y), 1)

In [0]:
### train, test 데이터로 분할 ###
train_idx = random.sample(list(range(len(credit_card))), int(len(credit_card) * 0.7))
train_X = credit_X[train_idx, :]
train_y = credit_y[train_idx]

test_X = np.delete(credit_X, train_idx, axis = 0)
test_y = np.delete(credit_y, train_idx)
test_y.shape = (len(test_y), 1)

In [46]:
### 모델 생성 ###
def build_model():
  model = keras.Sequential([
    layers.Dense(3, activation=tf.nn.relu, input_shape=[train_X.shape[1]]),
    layers.Dense(1),
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model


model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 3)                 36        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 4         
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


In [0]:
### 모델 훈련 ###

EPOCHS = 1000

history = model.fit(
  train_X, train_y,
  epochs=EPOCHS, validation_split = 0.2, verbose=0)

In [44]:
## 모델 예측 ###

test_predictions = model.predict(test_X)

loss, mae, mse = model.evaluate(test_X, test_y, verbose=0)
print("Testing set Mean Abs Error: {:5.2f}".format(mae))


Testing set Mean Abs Error:  0.24
